This file, once complete, will obtain a dataframe that contains every trade that occured during the 2022-23 NHL season. I am going to scrape the CapFriendly Website using beautiful soup and selenium and then use pandas to get it into the dataframe.

In [1]:
import requests
import pandas as pd
import time
import logging
import lxml
from selenium import webdriver
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

Now that necessary packages are installed, going to parse the needed url. This URL has a dynamic table that contains the cost per save of NHL Goalies for the 2020-21 season. So I am going to scrape the cap friendly website (Cost Per Save tab) and get this into a dataframe that I can eventually download and put into the MySQL database. As mentioned, since the table is dynamic and not static, the selenium package is needed because the table only expands to its full length once you scroll down.

In [2]:
# Set up the Selenium driver (make sure you have the correct path to your WebDriver)
driver_path = 'C:/STL Blues Analytics/SalaryCap/NHLSalaryCap/chromedriver.exe'  # Update this to your WebDriver path
service = Service(driver_path)
driver = webdriver.Chrome(service=service)

# Open the webpage
url = "https://www.capfriendly.com/cost-per-save/2021/season"
driver.get(url)

# Get initial page height
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to the bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    # Wait for new content to load
    time.sleep(1)
    
    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    
    if new_height == last_height:
        break
    last_height = new_height

# Get the page source after all rows are loaded
page_source = driver.page_source
driver.quit()

# Parse the page source with BeautifulSoup
soup = BeautifulSoup(page_source, 'html.parser')

# Find the table and convert it to a DataFrame
table = soup.find('table')
dfattempt = pd.read_html(str(table))[0]



C:\Users\nsofi\AppData\Local\Temp\ipykernel_12104\2115949715.py:36: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  dfattempt = pd.read_html(str(table))[0]


In [3]:
dfattempt.shape
dfattempt.tail(5)

,#,GOALIE,AGE,TYPE,EXPIRY,CAP HIT,P. BONUSES,GP,MIN,SH,SV,SV%,GAA,$/GP,$/MIN,$/SV
74,75,"Quick, Jonathan",34,NaN,Stnd,UFA,"$5,800,000",22,1218,566,508.0,0.898,2.86,"$263,636","$4,761","$11,417"
75,76,"Raanta, Antti",31,NaN,Stnd,UFA,"$4,250,000",12,678,401,363.0,0.905,3.36,"$354,166","$6,268","$11,707"
76,77,"Bobrovsky, Sergei",32,NaN,Stnd,UFA,"$10,000,000",31,1816,935,847.0,0.906,2.91,"$322,580","$5,506","$11,806"
77,78,"Rask, Tuukka",33,NaN,Stnd,UFA,"$7,000,000",24,1396,611,558.0,0.913,2.28,"$291,666","$5,014","$12,544"
78,79,"Price, Carey",33,NaN,Stnd,UFA,"$10,500,000",25,1478,659,594.0,0.901,2.64,"$420,000","$7,104","$17,676"


In [4]:
dfattempt.to_csv("CF Cost Per Save 2020-21.csv") 